# 第3章 字典和集合

字典和集合的实现均依赖于散列表，想要进一步理解集合和字典，就得先理解散列表的原理。

本章的内容大纲如下:

- 常见的字典方法

- 如何处理查找不到的键

- 标准库中dict类型的变种

- set和frozenset类型

- 散列表的工作原理

- 散列表带来的潜在影响（什么样的数据类型可作为键、不可预知的顺序，等等）

## 3.1 泛映射类型

collections.abc模块中有Mapping和MutableMapping这两个抽象基类，其作用在于为dict和其他类似的类型定义形式接口。

非抽象映射类型一般不会直接继承这些抽象基类，而是直接对dict或collections.User.Dict进行扩展。

In [ ]:
# 使用抽象基类和isinstance一起判定某个数据结构是否为广义的映射类型
import collections.abc as abc
mydict = {}
isinstance(mydict, abc.Mapping)

标准库里所有的映射类型都是利用dict来实现的，因此他们有个共同的限制，即只有可散列的数据类型才能用作这些映射里的键。

*<b>什么是可散列的数据类型？</b>*

<u>如果一个对象是可散列的，那么在这个对象的生命周期中，它的散列值是不变的，而且这个对象需要实现__hash__()方法。另外可散列的对象还要有__qe__()方法，这样才能跟其他键作比较。</u>

如果两个可散列对象是相等的，那么他们的散列值一定是一样的。

In [ ]:
tt = (1, 2, (30, 40))
hash(tt)

In [ ]:
tl = (1, 2, [30, 40])
hash(tl)

In [ ]:
tf = (1, 2, frozenset([30, 40]))
hash(tf)

In [ ]:
# 字典的构造方法
a = dict(one=1, two=2, three=3)
b = dict({'one':1, "two":2, "three":3})
c = dict(zip(['one', 'two', 'three'], [1, 2, 3]))
d = dict([('one', 1), ('two', 2), ('three', 3)])
e = {'one':1, "two":2, "three":3}

a == b == c == d == e

## 3.2 字典推导

字典推导（dictcomp）可以从任何以键值对作为元素的可迭代对象中构建出字典。

In [ ]:
# 字典推导的应用
DIAL_CODES = [ 
    (86, 'China'),
    (91, 'India'),
    (1, 'United States'),
    (62, 'Indonesia'),
    (55, 'Brazil'),
    (92, 'Pakistan'),
    (880, 'Bangladesh'), 
    (234, 'Nigeria'),
    (7, 'Russia'),
    (81, 'Japan'),
]
country_code = {country: code for code, country in DIAL_CODES}
country_code


In [ ]:
{code: country.upper() for country, code in country_code.items() if code < 56}

## 3.3 常见的映射方法

- 用 **setdefault** 处理找不到的键 

以下程序从索引中获取单词出现的频率信息，并把它们写进对应的列表里

In [ ]:
'''创建一个从单词到期出现情况的映射'''

import sys
import re

WORD_RE = re.compile(r'\w+')

index = {}
with open(sys.argv[1], encoding = 'utf-8') as fp:
    for line_no, line in enumerate(fp, 1):
        for match in WRRD_RE.finditer(line):
            word = match.group()
            column_no = match.start()+1
            location = (line_no, column_no)
            # 这其实是一种很不好的实现，这样写只是为了证明论点
            occurrences = index.get(word, []) ➊
            occurrences.append(location) ➋
            index[word] = occurrences ➌
# 以字母顺序打印出结果
for word in sorted(index, key=str.upper): ➍
    print(word, index[word])

In [ ]:
❶ 提取 word 出现的情况，如果还没有它的记录，返回 []。
❷ 把单词新出现的位置添加到列表的后面。
❸ 把新的列表放回字典中，这又牵扯到一次查询操作。
❹ sorted 函数的 key= 参数没有调用 str.uppper，而是把这个方法
的引用传递给 sorted 函数，这样在排序的时候，单词会被规范成统一
格式。
这是将方法用作一等函数的一个示例，第 5 章会谈到这一点。

# 此处插入 **正则表达式** 内容和 **enumerate** 函数。

## 1. enumerate函数

enumerate函数用于将一个可遍历的数据对象组合为一个索引序列，同时列出数据下标和数据，一般用在for循环中。

### 语法

`enumerate(sequence, [start=0])`

### 参数

sequence为可迭代序列，start为下标起始位置。

## 2. 正则表达式

- 正则表达式是一个特殊的字符序列，它能帮助你方便的检查一个字符串是否与某种模

式匹配。
- re模块使python语言拥有全部的正则表达式功能。

- compile函数根据一个模式字符串和可选的标志参数生成一个正则表达式对象。该对

象拥有一系列方法用于正则表达式匹配和替换。

### (1) **re.match** 函数

`re.match` 尝试从字符串的起始位置匹配一个模式，如果不是起始位置匹配成功的话，match就返回none.

#### 函数语法
`re.match(pattern, string, flags=0)`
#### 函数参数说明
| 参数 | 描述 |
|:---|:---:|
| pattern | 匹配的正则表示 |
| string  | 要匹配的字符串 |
| flags   | 	标志位，用于控制正则表达式的匹配方式，如：是否区分大小写，多行匹配等等。|

匹配成功re.match返回一个匹配的对象，否则返回None。

我们可以使用 `group(num)` 或 `groups()` 匹配对象函数来获取匹配表达式。

| 匹配对象方法 | 描述 |
|:---|:---|
|group(num=0)|匹配的整个表达式的字符串，group() 可以一次输入多个组号，在这种情况下它将返回一个包含那些组所对应值的元组。|
|groups|返回一个包含所有小组字符串的元组，从 1 到 所含的小组号。|



In [ ]:
#!/usr/bin/python
import re
 
line = "Cats are smarter than dogs"
 
matchObj = re.match( r'(.*) are (.*?) .*', line, re.M|re.I)
 
if matchObj:
    print ("matchObj.group() : ", matchObj.group())
    print ("matchObj.group(1) : ", matchObj.group(1))
    print ("matchObj.group(2) : ", matchObj.group(2))
    print(matchObj.groups())
else:
    print ("No match!!")

In [ ]:
matchObj.group()

### (2) ** re.search **函数
re.search扫描整个字符串并返回第一个成功匹配。

#### 函数语法
`re.search(pattern, string, flags=0)`
#### 参数说明
同上

### re.match与re.search的区别
re.match只匹配字符串的开始，如果字符串开始不符合正则表达式，则匹配失败，函数

返回None；而re.search匹配整个字符串，直到找到一个匹配。

In [ ]:
#!/usr/bin/python
import re
 
line = "Cats are smarter than dogs";
 
matchObj = re.match( r'dogs', line, re.M|re.I)
if matchObj:
    print ("match --> matchObj.group() : ", matchObj.group())
else:
    print ("No match!!")
 
matchObj = re.search( r'dogs', line, re.M|re.I)
if matchObj:
    print ("search --> searchObj.group() : ", matchObj.group())
else:
    print ("No match!!")

### （3） 检索和替换
python的re模块提供了re.sub用于替换字符串中的匹配项。
### 语法
`re.sub(pattern, repl, string, count=0, flags=0)`
### 参数
- pattern: 正则中的模式字符串
- repl: 替换的字符串，也可为一个函数
- string: 要被查找替换的原始字符串
- count: 模式匹配后替换的最大次数，默认0表示替换所有的匹配

In [ ]:
#!/usr/bin/python
# -*- coding: UTF-8 -*-
 
import re
 
phone = "2004-959-559 # 这是一个国外电话号码"
 
# 删除字符串中的 Python注释 
num = re.sub(r'#.*$', "", phone)
print ("电话号码是: ", num)
 
# 删除非数字(-)的字符串 
num = re.sub(r'\D', "", phone)
print ("电话号码是 : ", num)

#### repl参数是一个函数
以下实例中将字符串中的匹配的数字乘以2：

In [46]:
#!/usr/bin/python
# -*- coding: UTF-8 -*-
 
import re
 
# 将匹配的数字乘以 2
def double(matched):
    value = int(matched.group('value'))
    return str(value * 2)
 
s = 'A23G4HFD567'
print(re.sub('(?P<value>\d+)', double, s))

A46G8HFD1134


### （4）** re.compile **函数

compile函数用于编译正则表达式，生成一个正则表达式（Pattern）对象，供match和 

search这两个函数使用。

### 语法格式

`re.compile(pattern[, flags])`

### 参数

- pattern: 一个字符串形式的正则表达式

- flags: 可选，表示匹配模式，比如忽略大小写，多行模式等，具体参数为：

1. re.I 忽略大小写

2. re.L 表示特殊字符集 \w, \W, \b, \B, \s, \S 依赖于当前环境

3. re.M 多行模式

4. re.S 即为 `.`并且包括换行符在内的任意字符（`.` 不包括换行符）

5. re.U 表示特殊字符集 \w, \W, \b, \B, \d, \D, \s, \S 依赖于 Unicode 字符属性数据库

6. re.X 为了增加可读性，忽略空格和 # 后面的注释

In [ ]:
>>>import re
>>> pattern = re.compile(r'\d+')                    # 用于匹配至少一个数字
>>> m = pattern.match('one12twothree34four')        # 查找头部，没有匹配
>>> print m
None
>>> m = pattern.match('one12twothree34four', 2, 10) # 从'e'的位置开始匹配，没有匹配
>>> print m
None
>>> m = pattern.match('one12twothree34four', 3, 10) # 从'1'的位置开始匹配，正好匹配
>>> print m                                         # 返回一个 Match 对象
<_sre.SRE_Match object at 0x10a42aac0>
>>> m.group(0)   # 可省略 0
'12'
>>> m.start(0)   # 可省略 0
3
>>> m.end(0)     # 可省略 0
5
>>> m.span(0)    # 可省略 0
(3, 5)

在上面，当匹配成功时返回一个 Match 对象，其中：

- group([group1, …]) 方法用于获得一个或多个分组匹配的字符串，当要获得整个匹配的子串时，可直接使用 group() 或 group(0)；

- start([group]) 方法用于获取分组匹配的子串在整个字符串中的起始位置（子串第一个字符的索引），参数默认值为 0；

- end([group]) 方法用于获取分组匹配的子串在整个字符串中的结束位置（子串最后一个字符的索引+1），参数默认值为 0；

- span([group]) 方法返回 (start(group), end(group))。

In [ ]:
>>>import re
>>> pattern = re.compile(r'([a-z]+) ([a-z]+)', re.I)   # re.I 表示忽略大小写
>>> m = pattern.match('Hello World Wide Web')
>>> print m                               # 匹配成功，返回一个 Match 对象
<_sre.SRE_Match object at 0x10bea83e8>
>>> m.group(0)                            # 返回匹配成功的整个子串
'Hello World'
>>> m.span(0)                             # 返回匹配成功的整个子串的索引
(0, 11)
>>> m.group(1)                            # 返回第一个分组匹配成功的子串
'Hello'
>>> m.span(1)                             # 返回第一个分组匹配成功的子串的索引
(0, 5)
>>> m.group(2)                            # 返回第二个分组匹配成功的子串
'World'
>>> m.span(2)                             # 返回第二个分组匹配成功的子串
(6, 11)
>>> m.groups()                            # 等价于 (m.group(1), m.group(2), ...)
('Hello', 'World')
>>> m.group(3)                            # 不存在第三个分组
Traceback (most recent call last):
  File "<stdin>", line 1, in <module>
IndexError: no such groupb  

### （5） ** findall **

在字符串中找到正则表达式所匹配的所有子串，并返回一个列表，如果没有找到匹配的，则返回空列表。

** 注意 **

match和search是匹配一次，findall匹配所有。

### 语法格式

`findall(string[, pos[, endpos]])`

### 参数

- string : 待匹配的字符串。

- pos : 可选参数，指定字符串的起始位置，默认为 0。

- endpos : 可选参数，指定字符串的结束位置，默认为字符串的长度。


In [39]:
# -*- coding:UTF8 -*-
 
import re
 
pattern = re.compile(r'\d+')   # 查找数字
result1 = pattern.findall('runoob 123 google 456')
result2 = pattern.findall('run88oob123google456', 0, 10)
 
print(result1)
print(result2)

['123', '456']
['88', '12']


### （6）** re.finditer**

和 findall 类似，在字符串中找到正则表达式所匹配的所有子串，并把它们作为一个迭代器返回

`re.finditer(pattern, string, flags=0)`

In [43]:
# -*- coding: UTF-8 -*-
 
import re
 
it = re.finditer(r"\d+","12a32bc43jf3") 
for match in it: 
    print (match.group() )

12
32
43
3


### (7) **re.spilt**

split方法按照能够匹配的子串将字符串分割后返回列表，它的使用形式如下：

`re.spilt(pattern, string[,maxsplit=0, flags=0])`

maxsplit 分割次数，maxsplit=1分隔1次，默认为0，不限制次数

In [ ]:
>>>import re
>>> re.split('\W+', 'runoob, runoob, runoob.')
['runoob', 'runoob', 'runoob', '']
>>> re.split('(\W+)', ' runoob, runoob, runoob.') 
['', ' ', 'runoob', ', ', 'runoob', ', ', 'runoob', '.', '']
>>> re.split('\W+', ' runoob, runoob, runoob.', 1) 
['', 'runoob, runoob, runoob.']
 
>>> re.split('a*', 'hello world')   # 对于一个找不到匹配的字符串而言，split 不会对其作出分割
['hello world']

### 正则表达式对象

- `re.RegexObject`

re.compile()返回RegexObject对象

- `re.MatchObject`

group() 返回被 RE 匹配的字符串。

- start() 返回匹配开始的位置

- end() 返回匹配结束的位置

- span() 返回一个元组包含匹配 (开始,结束) 的位置

正则表达式模式等相关细节使用时另请详查。

# 下面返回FluentPython相关章节。

使用dict.setdefault使用一行代码解决获取和更新单词出现情况

In [45]:
import sys
import re

WORD_RE = re.compile(r'\w+')

index = {}
with open("./zen.txt", encoding='utf-8') as fp:
    for line_no, line in enumerate(fp, 1):
        for match in WORD_RE.finditer(line):
            word = match.group()
            column_no = match.start()+1
            location = (line_no, column_no)
            index.setdefault(word,[]).append(location)
# 以字母顺序打印出结果
for word in sorted(index, key=str.upper):
    print(word, index[word])


are [(1, 5)]
Big [(3, 1)]
Dipper [(3, 5)]
in [(4, 1)]
life [(4, 13)]
like [(2, 1)]
my [(4, 4)]
shining [(3, 13)]
the [(2, 6)]
whole [(4, 7)]
You [(1, 1)]


二者的效果是一样的，只不过后者至少要进行两次键查询——如果键不存在的话，就是三次，用 setdefault 只需要一次就可以完成整个操作。


那么，在单纯地查找取值（而不是通过查找来插入新值）的时候，该怎么处理找不到的键呢?

## 3.4 映射的弹性键查询

### 3.4.1 defaultdict: 处理找不到的键的一个选择

在实例化一个 defaultdict 的时候，需要给构造方法提供一个可调用对象，这个可调用对象会在 __getitem__ 碰到找不到的键的时候被调用，让 __getitem__ 返回某种默认值。



In [2]:
'''创建一个从单词到其出现情况的映射'''
import sys
import re
import collections

WORD_RE = re.compile(r'\w+')

index = collections.defaultdict(list)
with open("./zen.txt", encoding="utf-8") as fp:
    for line_no, line in enumerate(fp, 1):
        for match in WORD_RE.finditer(line):
            word = match.group()
            column_no = match.start() + 1
            location = (line_no,column_no)
            index[word].append(location)
# 以字母顺序打印出结果
for word in sorted(index, key=str.upper):
    print(word, index[word])



are [(1, 5)]
Big [(3, 1)]
Dipper [(3, 5)]
in [(4, 1)]
life [(4, 13)]
like [(2, 1)]
my [(4, 4)]
shining [(3, 13)]
the [(2, 6)]
whole [(4, 7)]
You [(1, 1)]


- 把 list 构造方法作为 default_factory 来创建一个
defaultdict。
- 如果 index 并没有 word 的记录，那么 default_factory 会被调
用，为查询不到的键创造一个值。这个值在这里是一个空的列表，然后
这个空列表被赋值给 index\[word\]，继而被当作返回值返回，因此
.append(location) 操作总能成功。
如果在创建 defaultdict 的时候没有指定 default_factory，查询不
存在的键会触发 KeyError。

> defaultdict 里的 default_factory 只会在
__getitem__ 里被调用，在其他的方法里完全不会发挥作用。比
如，dd 是个 defaultdict，k 是个找不到的键， dd[k] 这个表达
式会调用 default_factory 创造某个默认值，而 dd.get(k) 则会
返回 None。


所有这一切背后的功臣其实是特殊方法 __missing__。它会在defaultdict 遇到找不到的键的时候调用 default_factory，而实际上这个特性是所有映射类型都可以选择去支持的。

### 3.4.2 特殊方法\__missing\__

所有的映射类型在处理找不到的键的时候，都会牵扯到 __missing__方法。这也是这个方法称作“missing”的原因。虽然基类 dict 并没有定义这个方法，但是 dict 是知道有这么个东西存在的。也就是说，如果有一个类继承了 dict，然后这个继承类提供了 __missing__ 方法，那么在 __getitem__ 碰到找不到的键的时候，Python 就会自动调用它，而不是抛出一个 KeyError 异常。

***  __missing__ 方法只会被 __getitem__ 调用（比如在表达式 d\[k\] 中）。提供 __missing__ 方法对 get 或者__contains__（in 运算符会用到这个方法）这些方法的使用没有影响。这也是我在上一节最后的警告中提到，defaultdict 中default_factory 只对 __getitem__ 有作用的原因。 ***



In [3]:
# 以下示例实现了StrKeyDict0类的实现及当有非字符串的键被查找的时候，该类是如何在该键不存在的情况下，把它转换为字符串的
class StrKeyDict0(dict):

    def __missing__(self, key):
        if isinstance(key, str):
            raise KeyError(key)
        return self[str(key)]
    def get(self, key, default=None):
        try:
            return self[key]
        except KeyError:
            return default
    def __contains__(self, key):
        return key in self.keys() or str(key) in self.keys()
        
    

In [4]:
d = StrKeyDict0([('2', 'two'), ('4', 'four')])
print(d['2'], d[4])

two four


In [5]:
print(d[1])

KeyError: '1'

In [7]:
d.get(1,'ok')

'ok'

In [8]:
2 in d

True

## 3.5 字典的变种

1. collections.OrderedDict
这个类型在添加键的时候会保持顺序，因此键的迭代次序总是一致的。OrderedDict 的 popitem 方法默认删除并返回的是字典里的最后一个元素，但是如果像 my_odict.popitem(last=False) 这样调用它，那么它删除并返回第一个被添加进去的元素。
2. collections.ChainMap
该类型可以容纳数个不同的映射对象，然后在进行键查找操作的时候，这些对象会被当作一个整体被逐个查找，直到键被找到为止。这个功能在给有嵌套作用域的语言做解释器的时候很有用，可以用一个映射对象来代表一个作用域的上下文。
3. collections.Counter
这个映射类型会给键准备一个整数计数器。每次更新一个键的时候都会增加这个计数器。所以这个类型可以用来给可散列表对象计数，或者是当成多重集来用——多重集合就是集合里的元素可以出现不止一次。Counter 实现了 + 和 - 运算符用来合并记录，还有像most_common([n]) 这类很有用的方法。most_common([n]) 会按照次序返回映射里最常见的 n 个键和它们的计数.
4. collections.UserDict
这个类其实就是把标准 dict 用纯 Python 又实现了一遍。跟 OrderedDict、ChainMap 和 Counter 这些开箱即用的类型不同，UserDict 是让用户继承写子类的。下面就来试试。


In [10]:
# 利用Counter来计算单词中各个字母出现的次数
ct = collections.Counter('abrajsfdlsjdfgifg')
print(ct)
ct.update('aaaaazzz')
print(ct)
print(ct.most_common(2))

Counter({'f': 3, 'a': 2, 'j': 2, 's': 2, 'd': 2, 'g': 2, 'b': 1, 'r': 1, 'l': 1, 'i': 1})
Counter({'a': 7, 'f': 3, 'z': 3, 'j': 2, 's': 2, 'd': 2, 'g': 2, 'b': 1, 'r': 1, 'l': 1, 'i': 1})
[('a', 7), ('f', 3)]


## 3.6 子类化UserDict
